In [1]:
  ! pip install -r test1.txt

In [2]:
pip install pyspark

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
from pyspark.sql import SparkSession , functions as F

spark = SparkSession.builder.appName('Crime')\
    .config('spark.driver.extraClassPath', '/usr/lib/jvm/java-11-openjdk-amd64/lib/postgresql-42.5.0.jar')\
    .getOrCreate()

22/11/03 17:35:29 WARN Utils: Your hostname, shijal-Lenovo-IdeaPad-S145-15IWL resolves to a loopback address: 127.0.1.1; using 192.168.101.12 instead (on interface wlp0s20f3)
22/11/03 17:35:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/03 17:35:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/11/03 17:35:30 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
police_df=spark.read.csv('police_district_code.csv')

In [5]:
police_df.show()

+-------------+-------------+
|          _c0|          _c1|
+-------------+-------------+
|District_Code|District_Name|
|           A1|     Downtown|
|          A15|  Charlestown|
|           A7|  East Boston|
|           B2|      Roxbury|
|           B3|     Mattapan|
|           C6| South Boston|
|          C11|   Dorchester|
|           D4|    South End|
|          D14|     Brighton|
|           E5| West Roxbury|
|          E13|Jamaica Plain|
|          E18|    Hyde Park|
+-------------+-------------+



In [6]:
Properties={'user': 'shijal', 'password': 'shijal@123'}
URL='jdbc:postgresql://localhost:5432/postgres'

In [7]:
police_df.write.jdbc(url=URL, table='police', mode='overwrite', properties=Properties)

In [9]:
crime_df=spark.read.csv('crimes.csv')

In [11]:
crime_df.show()

+---------------+------------+--------------------+--------------------+--------+--------------+--------+-------------------+----+-----+-----------+----+----------+-----------------+-----------+------------+--------------------+
|            _c0|         _c1|                 _c2|                 _c3|     _c4|           _c5|     _c6|                _c7| _c8|  _c9|       _c10|_c11|      _c12|             _c13|       _c14|        _c15|                _c16|
+---------------+------------+--------------------+--------------------+--------+--------------+--------+-------------------+----+-----+-----------+----+----------+-----------------+-----------+------------+--------------------+
|INCIDENT_NUMBER|OFFENSE_CODE|  OFFENSE_CODE_GROUP| OFFENSE_DESCRIPTION|DISTRICT|REPORTING_AREA|SHOOTING|   OCCURRED_ON_DATE|YEAR|MONTH|DAY_OF_WEEK|HOUR|  UCR_PART|           STREET|        Lat|        Long|            Location|
|     I182070945|       00619|             Larceny|  LARCENY ALL OTHERS|     D14|   

In [14]:
crimes_df = spark.read.csv('/home/shijal/Desktop/Crime/crimes.csv', header=True, inferSchema=True)
offense_codes_df = spark.read.csv('/home/shijal/Desktop/Crime/offense_codes.csv', header=True, inferSchema=True)
police_district_codes_df = spark.read.csv('/home/shijal/Desktop/Crime/police_district_code.csv', header=True, inferSchema=True)

In [15]:
# for SHOOTING column replace null values with 'N'
crimes_df = crimes_df.withColumn('SHOOTING', F.when(F.col('SHOOTING').isNull(), 'N')\
    .otherwise(F.col('SHOOTING')))

# remove OFFENSE_DESCRIPTION column
crimes_df = crimes_df.drop('OFFENSE_DESCRIPTION')
crimes_df.show(5)

# keep only first duplicate value in offence_code_df
offense_codes_df = offense_codes_df.dropDuplicates(['CODE'])
offense_codes_df.sort ('CODE').show(5)

+---------------+------------+--------------------+--------+--------------+--------+-------------------+----+-----+-----------+----+----------+-----------+-----------+------------+--------------------+
|INCIDENT_NUMBER|OFFENSE_CODE|  OFFENSE_CODE_GROUP|DISTRICT|REPORTING_AREA|SHOOTING|   OCCURRED_ON_DATE|YEAR|MONTH|DAY_OF_WEEK|HOUR|  UCR_PART|     STREET|        Lat|        Long|            Location|
+---------------+------------+--------------------+--------+--------------+--------+-------------------+----+-----+-----------+----+----------+-----------+-----------+------------+--------------------+
|     I182070945|         619|             Larceny|     D14|           808|       N|2018-09-02 13:00:00|2018|    9|     Sunday|  13|  Part One| LINCOLN ST|42.35779134|-71.13937053|(42.35779134, -71...|
|     I182070943|        1402|           Vandalism|     C11|           347|       N|2018-08-21 00:00:00|2018|    8|    Tuesday|   0|  Part Two|   HECLA ST|42.30682138|-71.06030035|(42.30682138

11. Find the average number of crime happened  for each year.

In [60]:
crimes_df.groupBy("Year").count().show()
################### SAVE to POSTGRES #######################
crimes_df.write.jdbc(url=URL, table='YEAR', mode='overwrite', properties=Properties)


+----+------+
|Year| count|
+----+------+
|2018| 65685|
|2015| 53388|
|2016| 99114|
|2017|100886|
+----+------+



12. How many crimes were committed in 2018.

In [50]:
total_2018 = crimes_df.where((crimes_df.YEAR==2018)).count()
print("Total crimes committed in 2018 is:",total_2018)
################### SAVE to POSTGRES #######################
crimes_df.write.jdbc(url=URL, table='YEAR', mode='overwrite', properties=Properties)

Total crimes committed in 2018 is: 65685


13. Find how many times ‘Auto Theft’ happened in each year.

In [86]:
filtering_autotheft = crimes_df.filter((crimes_df.OFFENSE_CODE_GROUP=="Auto Theft")).select(crimes_df.YEAR,crimes_df.OFFENSE_CODE_GROUP)
total_autotheft = filtering_autotheft.groupBy("YEAR").count().orderBy("Year").withColumnRenamed("count","Total Auto Theft in Year")
total_autotheft.show()
################### SAVE to POSTGRES #######################
crimes_df.write.jdbc(url=URL, table='YEAR', mode='overwrite', properties=Properties)

+----+------------------------+
|YEAR|Total Auto Theft in Year|
+----+------------------------+
|2015|                     988|
|2016|                    1537|
|2017|                    1393|
|2018|                     933|
+----+------------------------+



14. Reporting Area having highest Shooting incident. 

In [53]:
crimes_df.groupBy("Shooting").count().show()

+--------+------+
|Shooting| count|
+--------+------+
|       Y|  1019|
|       N|318054|
+--------+------+



15. Arrange street based on high “Rape” incident